<a href="https://colab.research.google.com/github/AishlyManglani/Deep-Learning_-CMPE258/blob/main/computer%20vision/3d_image_classication_of_ctscan_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# ============================================
# 1. Install & Import Libraries
# ============================================
!pip install -q tensorflow tensorflow_datasets

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np


In [5]:
# ============================================
# 2. Create Synthetic 3D Dataset
# ============================================
def create_synthetic_3d_data(num_samples=100, img_size=(64, 64, 64)):
    X = []
    y = []
    for i in range(num_samples):
        # Create random 3D volume
        volume = np.random.rand(*img_size).astype(np.float32)
        label = np.random.randint(0, 2)  # Binary classes
        X.append(volume)
        y.append(label)
    return np.expand_dims(np.array(X), -1), np.array(y)  # Add channel dim

# Create training and validation sets
X_train, y_train = create_synthetic_3d_data(200)
X_val, y_val = create_synthetic_3d_data(50)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")

X_train shape: (200, 64, 64, 64, 1), y_train shape: (200,)


In [6]:
# ============================================
# 3. Build a 3D ConvNet Model
# ============================================
inputs = tf.keras.Input(shape=(64, 64, 64, 1))

x = tf.keras.layers.Conv3D(32, 3, activation='relu')(inputs)
x = tf.keras.layers.MaxPooling3D(2)(x)
x = tf.keras.layers.Conv3D(64, 3, activation='relu')(x)
x = tf.keras.layers.GlobalAveragePooling3D()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs, outputs)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 64, 64, 64, 1)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d (Conv3D)                 │ (None, 62, 62, 62, 32) │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d (MaxPooling3D)    │ (None, 31, 31, 31, 32) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_1 (Conv3D)               │ (None, 29, 29, 29, 64) │        55,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling3d        │ (None, 64)             │             0 │
│ (GlobalAveragePooling3D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 60,481 (236.25 KB)

 Trainable params: 60,481 (236.25 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# ============================================
# 4. Train Model
# ============================================
EPOCHS = 5
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=8
)


Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 120ms/step - accuracy: 0.4237 - loss: 0.7022 - val_accuracy: 0.3600 - val_loss: 0.7112
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.5351 - loss: 0.6920 - val_accuracy: 0.3600 - val_loss: 0.7128
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.5074 - loss: 0.6954 - val_accuracy: 0.3600 - val_loss: 0.7211
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - accuracy: 0.5350 - loss: 0.6950 - val_accuracy: 0.3600 - val_loss: 0.7108
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.5304 - loss: 0.6918 - val_accuracy: 0.3600 - val_loss: 0.7199


In [8]:
# ============================================
# 5. Evaluate Model
# ============================================
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"✅ Validation accuracy on synthetic 3D dataset: {val_acc:.2f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 956ms/step - accuracy: 0.3129 - loss: 0.7278
✅ Validation accuracy on synthetic 3D dataset: 0.36
